In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from math import sqrt, ceil
from timeit import default_timer as timer

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Open data file
with open("./dataset/data2.pickle", 'rb') as f:
    data = pickle.load(f, encoding="latin1")

# Prepare for keras
data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

# Making channels come at the end
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

# Showing loaded data from file
for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else: 
        print(i + ':', j.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=5, padding='same', activation='relu', input_shape=(32, 32, 3)))
BatchNormalization(),
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(256, kernel_size=5, padding='same', activation='relu', input_shape=(32, 32, 3)))
BatchNormalization(),
model.add(MaxPool2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
epochs = 10


h= model.fit(data['x_train'], data['y_train'],
                    batch_size=500, epochs = epochs,
                    validation_data = (data['x_validation'], data['y_validation']),
                    callbacks=[annealer], verbose=1)

In [ ]:
plt.rcParams['figure.figsize'] = (15.0, 15.0) # Setting default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['font.family'] = 'Times New Roman'

# Plotting history of training accuracy
fig = plt.figure()
plt.plot(h.history['accuracy'], '-o', linewidth=3.0)
plt.plot(h.history['val_accuracy'], '-o', linewidth=3.0)
plt.legend(['train', 'validation'], loc='upper left', fontsize='xx-large')
plt.xlabel('Epoch', fontsize=20, fontname='Times New Roman')
plt.ylabel('Accuracy', fontsize=20, fontname='Times New Roman')
plt.yscale('linear')  # {"linear", "log", "symlog", "logit", ...}
plt.ylim(0.90, 1.0)
plt.xlim(0.5, 10.3) 
plt.title('Training and Testing Accuracy', fontsize=22)

# plt.subplot(2, 1, 2)
# # plt.gca().set_title('Validation accuracy')

# plt.plot(h.history['val_accuracy'], '-o', linewidth=3.0)
# plt.xlabel('Epoch', fontsize=20, fontname='Times New Roman')
# plt.ylabel('Validation Accuracy', fontsize=20, fontname='Times New Roman')
# plt.yscale('linear')  # {"linear", "log", "symlog", "logit", ...}
# plt.ylim(0.75, 0.999)
# plt.xlim(0, 10.3)

# Showing the plot
plt.show()

print('data2 filter {0:d} training accuracy = {1:.5f}'.\
      format(5, np.max(h.history['accuracy'])))

print()

print('data2 filter {0:d} validation accuracy = {1:.5f}'.\
      format(5, np.max(h.history['val_accuracy'])))

In [ ]:
temp = model.predict(data['x_test'])
temp = np.argmax(temp, axis=1)

# We compare predicted class with correct class for all input images
# And calculating mean value among all values of following numpy array
# By saying 'testing_accuracy == data['y_test']' we create numpy array with True and False values
# 'np.mean' function will return average of the array elements
# The average is taken over the flattened array by default
temp = np.mean(temp == data['y_test'])

print('data2 filter {0:d} testing accuracy = {1:.5f}'.format(5, temp))